In [3]:
!pip install pandas seaborn scikit-learn numpy

  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 34.5 MB/s eta 0:00:0000:0100:01
Using cached joblib-1.4.2-py3-none-any.whl (301 kB)
Using cached threadpoolctl-3.5.0-py3-none-any.whl (18 kB)


Falta quitar outliers y balancear pero no corresponde aun

In [2]:
import os
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
import numpy as np

## Dataset
El dataset se centra en el cancer de mama, un tipo de cáncer más comun entre mujeres a nivel mundial. La descripción resalta la importancia de realizar estimaciones precisas de pronóstico y duración de supervivencia en la toma de decisiones clínicas para pacientes con cáncer. 

Este dataset se centra en la expresión génica, comparando tejidos normales y enfermos.

Los datos en este estudio provienen de la base de datos METABRIC (Consorcio Internacional de Taxonomía Molecular del Cáncer de Mama), que incluye datos genómicos y clínicos de casi 2,000 muestras de cáncer de mama. Estos datos fueron recopilados como parte de una colaboración entre Canadá y el Reino Unido, y han sido ampliamente utilizados en investigaciones, incluyendo publicaciones de alto impacto.

## Carga de datos

In [3]:
raw_data = "raw_data/METABRIC_RNA_Mutation.csv"

In [4]:
df = pd.read_csv(raw_data)
df

/tmp/ipykernel_737151/3204783994.py:1: DtypeWarning: Columns (678,688,690,692) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_data)


,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,7295,43.10,BREAST CONSERVING,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,LumA,4.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1900,7296,42.88,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1901,7297,62.90,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1902,7298,61.16,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,4.0,Positve,...,0,0,0,0,0,0,0,0,0,0


In [5]:
df.shape

(1904, 693)

## Atributos del dataset

### Listando todas las columnas

In [7]:
print(str(df.columns))

Index(['patient_id', 'age_at_diagnosis', 'type_of_breast_surgery',
       'cancer_type', 'cancer_type_detailed', 'cellularity', 'chemotherapy',
       'pam50_+_claudin-low_subtype', 'cohort', 'er_status_measured_by_ihc',
       ...
       'mtap_mut', 'ppp2cb_mut', 'smarcd1_mut', 'nras_mut', 'ndfip1_mut',
       'hras_mut', 'prps2_mut', 'smarcb1_mut', 'stmn2_mut', 'siah1_mut'],
      dtype='object', length=693)


## Descripcion de columnas

Existen 3 tipos de columnas:
- **Atributos clínicos:** Atributos que describen al paciente.

- **mRNA z-score:** Cálculo de expresión relativa de un gen individual y del tumor con respecto a la distribución de la expresión del gen en una población de referencia (todas las muestras del estudio). El valor devuelto indica el número de desviaciones estándar que se alejan de la media de expresión en la población de referencia (z-score). Se explica en el dataset que esta medida es útil para determinar si un gen esta regulado al alza o a la baja en relación con las muestras normales o con todas las demas muestras tumorales.

Cálculo:

z = (expression in tumor sample - mean expression in reference sample) / standard deviation of expression in reference sample

- **Mutaciones:** Indican el nombre de una mutacion en un gen.


### Atributos clínicos

In [8]:
clinical_attributes = ["patient_id", "age_at_diagnosis", "type_of_breast_surgery", "cancer_type", "cancer_type_detailed", "cellularity", "chemotherapy", "pam50_+_claudin-low_subtype", "cohort", "er_status_measured_by_ihc", "er_status", "neoplasm_histologic_grade", "her2_status_measured_by_snp6", "her2_status", "tumor_other_histologic_subtype", "hormone_therapy", "inferred_menopausal_state", "integrative_cluster", "primary_tumor_laterality", "lymph_nodes_examined_positive", "mutation_count", "nottingham_prognostic_index", "oncotree_code", "overall_survival_months", "overall_survival", "pr_status", "radio_therapy", "3-gene_classifier_subtype", "tumor_size", "tumor_stage", "death_from_cancer"]

In [9]:
clinical_attributes_df = df[clinical_attributes]
clinical_attributes_df

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,nottingham_prognostic_index,oncotree_code,overall_survival_months,overall_survival,pr_status,radio_therapy,3-gene_classifier_subtype,tumor_size,tumor_stage,death_from_cancer
0,0,75.65,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,NaN,0,claudin-low,1.0,Positve,...,6.044,IDC,140.500000,1,Negative,1,ER-/HER2-,22.0,2.0,Living
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,4.020,IDC,84.633333,1,Positive,1,ER+/HER2- High Prolif,10.0,1.0,Living
2,5,48.87,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,1,LumB,1.0,Positve,...,4.030,IDC,163.700000,0,Positive,0,NaN,15.0,2.0,Died of Disease
3,6,47.68,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,Moderate,1,LumB,1.0,Positve,...,4.050,MDLC,164.933333,1,Positive,1,NaN,25.0,2.0,Living
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,6.080,MDLC,41.366667,0,Positive,1,ER+/HER2- High Prolif,40.0,2.0,Died of Disease
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,7295,43.10,BREAST CONSERVING,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,LumA,4.0,Positve,...,5.050,ILC,196.866667,1,Positive,1,ER+/HER2- Low Prolif,25.0,NaN,Living
1900,7296,42.88,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,...,5.040,IDC,44.733333,0,Negative,1,NaN,20.0,NaN,Died of Disease
1901,7297,62.90,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumB,4.0,Positve,...,6.050,IDC,175.966667,0,Positive,1,NaN,25.0,NaN,Died of Disease
1902,7298,61.16,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,4.0,Positve,...,5.050,IDC,86.233333,0,Positive,0,ER+/HER2- High Prolif,25.0,NaN,Died of Other Causes


In [10]:
## Guardo este valor porque me va a servir para cuando haga ordinal encoding
ca_object_columns = []
for col in clinical_attributes_df.columns:
    if str(clinical_attributes_df[col].dtype) == "object":
        ca_object_columns.append(col)
ca_object_columns

['type_of_breast_surgery',
 'cancer_type',
 'cancer_type_detailed',
 'cellularity',
 'pam50_+_claudin-low_subtype',
 'er_status_measured_by_ihc',
 'er_status',
 'her2_status_measured_by_snp6',
 'her2_status',
 'tumor_other_histologic_subtype',
 'inferred_menopausal_state',
 'integrative_cluster',
 'primary_tumor_laterality',
 'oncotree_code',
 'pr_status',
 '3-gene_classifier_subtype',
 'death_from_cancer']

In [11]:
# Algunos grafiquitos por aqui, por allá...

### Columnas z-score

In [12]:
z_score_cols = [col for col in df.columns if col not in clinical_attributes and "mut" not in col ]
z_score_cols

['brca1',
 'brca2',
 'palb2',
 'pten',
 'tp53',
 'atm',
 'cdh1',
 'chek2',
 'nbn',
 'nf1',
 'stk11',
 'bard1',
 'mlh1',
 'msh2',
 'msh6',
 'pms2',
 'epcam',
 'rad51c',
 'rad51d',
 'rad50',
 'rb1',
 'rbl1',
 'rbl2',
 'ccna1',
 'ccnb1',
 'cdk1',
 'ccne1',
 'cdk2',
 'cdc25a',
 'ccnd1',
 'cdk4',
 'cdk6',
 'ccnd2',
 'cdkn2a',
 'cdkn2b',
 'myc',
 'cdkn1a',
 'cdkn1b',
 'e2f1',
 'e2f2',
 'e2f3',
 'e2f4',
 'e2f5',
 'e2f6',
 'e2f7',
 'e2f8',
 'src',
 'jak1',
 'jak2',
 'stat1',
 'stat2',
 'stat3',
 'stat5a',
 'stat5b',
 'mdm2',
 'tp53bp1',
 'adam10',
 'adam17',
 'aph1a',
 'aph1b',
 'arrdc1',
 'cir1',
 'ctbp1',
 'ctbp2',
 'cul1',
 'dll1',
 'dll3',
 'dll4',
 'dtx1',
 'dtx2',
 'dtx3',
 'dtx4',
 'ep300',
 'fbxw7',
 'hdac1',
 'hdac2',
 'hes1',
 'hes5',
 'heyl',
 'itch',
 'jag1',
 'jag2',
 'kdm5a',
 'lfng',
 'maml1',
 'maml2',
 'maml3',
 'ncor2',
 'ncstn',
 'notch1',
 'notch2',
 'notch3',
 'nrarp',
 'numb',
 'numbl',
 'psen1',
 'psen2',
 'psenen',
 'rbpj',
 'rbpjl',
 'rfng',
 'snw1',
 'spen',
 'hes2',


In [13]:
z_score_df = df[["patient_id"] + z_score_cols]
z_score_df

,patient_id,brca1,brca2,palb2,pten,tp53,atm,cdh1,chek2,nbn,...,srd5a1,srd5a2,srd5a3,st7,star,tnk2,tulp4,ugt2b15,ugt2b17,ugt2b7
0,0,-1.3990,-0.5738,-1.6217,1.4524,0.3504,1.1517,0.0348,0.1266,-0.8361,...,-1.1877,-0.0194,-1.6345,-0.2142,-0.5698,-1.1741,-1.4779,-0.5954,-0.8847,-0.3354
1,2,-1.3800,0.2777,-1.2154,0.5296,-0.0136,-0.2659,1.3594,0.7961,0.5419,...,-0.4412,0.4534,0.4068,0.7634,0.0231,0.9121,-0.9538,-0.2264,0.5398,-0.8920
2,5,0.0670,-0.8426,0.2114,-0.3326,0.5141,-0.0803,1.1398,0.4187,-0.4030,...,-0.5381,0.0668,0.8344,1.7227,0.4024,-3.7172,-1.5538,1.3701,-0.1078,0.3655
3,6,0.6744,-0.5428,-1.6592,0.6369,1.6708,-0.8880,1.2491,-1.1889,-0.4174,...,-0.5630,-0.7078,0.8228,0.6819,-0.1948,-2.3286,-0.9924,-0.3154,0.2320,-0.4828
4,8,1.2932,-0.9039,-0.7219,0.2168,0.3484,0.3897,0.9131,0.9356,0.7675,...,-0.5845,-0.3544,-1.0150,2.2961,0.1817,-0.1572,0.0427,5.0048,3.8476,1.3223
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,7295,0.1563,0.5543,-0.6149,0.4572,1.3822,-0.0537,-0.1323,0.2837,-0.0726,...,0.7859,1.5304,1.1196,0.8116,0.0958,-1.2644,-0.3566,-0.2615,-0.6371,-0.5540
1900,7296,0.1343,0.9128,1.3017,0.7383,0.1841,0.0028,0.1243,2.2040,-0.6190,...,-0.5294,-0.6610,-0.1610,0.2251,0.5127,-0.3412,-1.0703,-0.6950,0.0641,-0.8863
1901,7297,1.8107,-0.2608,0.4006,-0.2985,0.0356,-0.1620,1.5486,1.5309,-1.3653,...,-0.1457,-0.5194,2.3968,0.5917,-0.8643,-1.3541,-1.8076,-0.1840,0.0967,-0.6264
1902,7298,-1.2746,-1.7695,-0.3454,-0.3850,0.6689,1.4531,1.0956,-0.0948,-0.6188,...,-0.6380,-1.5275,1.1070,0.2194,1.4317,-0.6985,-0.6046,-0.4915,-0.2847,-0.8041


### Mutaciones en genes

In [14]:
mut_cols = [col for col in df.columns if "_mut" in col]
mut_cols

['pik3ca_mut',
 'tp53_mut',
 'muc16_mut',
 'ahnak2_mut',
 'kmt2c_mut',
 'syne1_mut',
 'gata3_mut',
 'map3k1_mut',
 'ahnak_mut',
 'dnah11_mut',
 'cdh1_mut',
 'dnah2_mut',
 'kmt2d_mut',
 'ush2a_mut',
 'ryr2_mut',
 'dnah5_mut',
 'herc2_mut',
 'pde4dip_mut',
 'akap9_mut',
 'tg_mut',
 'birc6_mut',
 'utrn_mut',
 'tbx3_mut',
 'col6a3_mut',
 'arid1a_mut',
 'lama2_mut',
 'notch1_mut',
 'cbfb_mut',
 'ncor2_mut',
 'col12a1_mut',
 'col22a1_mut',
 'pten_mut',
 'akt1_mut',
 'atr_mut',
 'thada_mut',
 'ncor1_mut',
 'stab2_mut',
 'myh9_mut',
 'runx1_mut',
 'nf1_mut',
 'map2k4_mut',
 'ros1_mut',
 'lamb3_mut',
 'arid1b_mut',
 'erbb2_mut',
 'sf3b1_mut',
 'shank2_mut',
 'ep300_mut',
 'ptprd_mut',
 'usp9x_mut',
 'setd2_mut',
 'setd1a_mut',
 'thsd7a_mut',
 'afdn_mut',
 'erbb3_mut',
 'rb1_mut',
 'myo1a_mut',
 'alk_mut',
 'fanca_mut',
 'adgra2_mut',
 'ubr5_mut',
 'pik3r1_mut',
 'myo3a_mut',
 'asxl2_mut',
 'apc_mut',
 'ctcf_mut',
 'asxl1_mut',
 'fancd2_mut',
 'taf1_mut',
 'kdm6a_mut',
 'ctnna3_mut',
 'brca1_mut

In [15]:
mut_df = df[["patient_id"] + mut_cols]
mut_df

,patient_id,pik3ca_mut,tp53_mut,muc16_mut,ahnak2_mut,kmt2c_mut,syne1_mut,gata3_mut,map3k1_mut,ahnak_mut,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,H178P,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,H1047R,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,E542K,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,S241F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,7295,E542K,0,0,0,0,0,0,R1482*,0,...,0,0,0,0,0,0,0,0,0,0
1900,7296,0,M237I,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,7297,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1902,7298,0,0,0,V3102M,0,0,S404Kfs*104 L94Pfs*117,0,R5779C,...,0,0,0,0,0,0,0,0,0,0


In [16]:
# No hay nulos
mut_df.dropna()

,patient_id,pik3ca_mut,tp53_mut,muc16_mut,ahnak2_mut,kmt2c_mut,syne1_mut,gata3_mut,map3k1_mut,ahnak_mut,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,2,0,H178P,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,5,H1047R,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,6,E542K,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,0,S241F,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1899,7295,E542K,0,0,0,0,0,0,R1482*,0,...,0,0,0,0,0,0,0,0,0,0
1900,7296,0,M237I,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1901,7297,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1902,7298,0,0,0,V3102M,0,0,S404Kfs*104 L94Pfs*117,0,R5779C,...,0,0,0,0,0,0,0,0,0,0


Existen mutaciones con nombres repetidos, por ejemplo "E542K", por lo que se podria aplicar un Ordinal Encoding junto con los otros atributos.

In [17]:
for col in mut_df.columns:
    print(mut_df[col].value_counts())

patient_id
7281    1
7280    1
7279    1
7278    1
7277    1
       ..
8       1
6       1
5       1
2       1
0       1
Name: count, Length: 1904, dtype: int64
pik3ca_mut
0               1109
H1047R           285
E545K            127
E542K             72
N345K             42
                ... 
N345K T727K        1
E103_V105del       1
N345I              1
A1035T             1
W11C               1
Name: count, Length: 160, dtype: int64
tp53_mut
0             1245
R175H           34
R248Q           20
R248W           17
R213*           15
              ... 
K132N E62*       1
K291*            1
H179D            1
P67Qfs*56        1
I255S            1
Name: count, Length: 343, dtype: int64
muc16_mut
0                1578
L7102F              5
T7149A              4
M6048T              3
D902A               3
                 ... 
E546K               1
S6804del            1
P113S T7510S        1
Q4733*              1
S5959F S6036C       1
Name: count, Length: 298, dtype: int64
ahnak2_mut

## Limpieza de nulos

In [18]:
export_df = df.dropna(axis=0, how="any")
export_df

,patient_id,age_at_diagnosis,type_of_breast_surgery,cancer_type,cancer_type_detailed,cellularity,chemotherapy,pam50_+_claudin-low_subtype,cohort,er_status_measured_by_ihc,...,mtap_mut,ppp2cb_mut,smarcd1_mut,nras_mut,ndfip1_mut,hras_mut,prps2_mut,smarcb1_mut,stmn2_mut,siah1_mut
1,2,43.19,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,LumA,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
4,8,76.97,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
5,10,78.77,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
8,28,86.41,BREAST CONSERVING,Breast Cancer,Breast Invasive Ductal Carcinoma,Moderate,0,LumB,1.0,Positve,...,0,0,0,0,0,0,0,0,0,0
9,35,84.22,MASTECTOMY,Breast Cancer,Breast Invasive Lobular Carcinoma,High,0,Her2,1.0,Negative,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1618,6232,71.22,MASTECTOMY,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,0,LumA,5.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1619,6233,70.65,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,0,LumB,5.0,Positve,...,0,0,0,0,0,0,0,0,0,0
1621,6237,75.62,MASTECTOMY,Breast Cancer,Breast Invasive Ductal Carcinoma,High,0,Basal,5.0,Negative,...,0,0,0,0,0,0,0,0,0,0
1623,6239,52.84,BREAST CONSERVING,Breast Cancer,Breast Mixed Ductal and Lobular Carcinoma,High,1,Normal,5.0,Positve,...,0,0,0,0,0,0,0,0,0,0


In [19]:
# Pierdo muchos datos al dropear nulos
original_length = df.shape[0]
tmp = df.dropna(axis=0, how="any")
print(f"Original Length: {original_length}\nAfter null drop length: {tmp.shape[0]}")

Original Length: 1904
After null drop length: 1092


In [20]:
# Se podria hacer algo aqui para evitar tanta perdida
nulls = df.isnull().sum()
nulls[nulls > 0].sort_values(ascending=False)

tumor_stage                       501
3-gene_classifier_subtype         204
primary_tumor_laterality          106
neoplasm_histologic_grade          72
cellularity                        54
mutation_count                     45
er_status_measured_by_ihc          30
type_of_breast_surgery             22
tumor_size                         20
cancer_type_detailed               15
tumor_other_histologic_subtype     15
oncotree_code                      15
death_from_cancer                   1
dtype: int64

In [21]:
# Por ahora no hice nada con la separacion de columnas asi que puedo reutilizar el df original.
export_df = export_df.drop(columns = ["patient_id"])

### Posibles columnas respuesta:

- **overall_survival_months:** Duration from the time of the intervention to death
- **overall_survival:** Target variable wether the patient is alive of dead.
- **death_from_cancer:** Wether the patient's death was due to cancer or not (yes/no)

Utilizaré overall_survival hasta que salga de la duda...

In [22]:
# Compruebo si no hay nulos
response = export_df["overall_survival"]
response

1       1
4       0
5       0
8       0
9       0
       ..
1618    0
1619    1
1621    0
1623    1
1664    1
Name: overall_survival, Length: 1092, dtype: int64

In [23]:
# Hay desbalance de clases
response.value_counts()

overall_survival
0    608
1    484
Name: count, dtype: int64

## Separación en ( train (80) + val (20) )[90] + test [10]

In [25]:
X_train, X_test, y_train, y_test = train_test_split(export_df, response, test_size=0.1, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

In [117]:
if not os.path.exists("data_separated"):
    os.mkdir("data_separated")

X_train.to_csv("data_separated/train_data.csv", index=False)
X_val.to_csv("data_separated/val_data.csv", index=False)
X_test.to_csv("data_separated/test_data.csv", index=False)
y_train.to_csv("data_separated/y_train.csv", index=False)
y_val.to_csv("data_separated/y_val.csv", index=False)
y_test.to_csv("data_separated/y_test.csv", index=False)

# DEPRECATED

In [51]:
survival_1 = df[df["overall_survival"] == 1]
survival_0 =resample(df[df["overall_survival"] == 0], n_samples=survival_1.shape[0])
df = pd.concat([survival_1, survival_0])
df

,age_at_diagnosis,chemotherapy,cohort,neoplasm_histologic_grade,hormone_therapy,lymph_nodes_examined_positive,mutation_count,nottingham_prognostic_index,overall_survival_months,overall_survival,...,srd5a1,srd5a2,srd5a3,st7,star,tnk2,tulp4,ugt2b15,ugt2b17,ugt2b7
1,43.19,0,1.0,3.0,1,0.0,2.0,4.020,84.633333,1,...,-0.4412,0.4534,0.4068,0.7634,0.0231,0.9121,-0.9538,-0.2264,0.5398,-0.8920
19,45.43,1,1.0,3.0,1,0.0,5.0,4.046,140.866667,1,...,1.8932,-0.5688,-1.3892,0.9588,-0.0417,-0.6983,-1.1513,-0.3737,-0.1071,-0.7333
22,61.49,0,1.0,2.0,1,1.0,3.0,4.032,157.433333,1,...,-1.2288,0.4227,-0.1570,0.6778,1.6169,-1.6327,0.9158,5.0569,-0.7728,2.8982
31,46.89,0,1.0,2.0,1,0.0,3.0,3.068,148.033333,1,...,0.7693,-1.0047,-0.3767,0.7846,0.2334,-1.4285,-1.4207,3.7802,-1.1134,2.7724
33,49.87,1,1.0,1.0,1,5.0,4.0,4.140,85.333333,1,...,-0.2678,1.9758,1.1248,0.0801,0.0935,-0.1223,-0.0312,-0.4591,-0.3754,-0.7703
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
905,57.98,0,3.0,3.0,1,9.0,5.0,6.030,111.566667,0,...,0.1539,0.4315,-0.0844,-1.1365,-0.4443,0.6097,2.0752,-0.1821,-0.2287,-0.8529
1246,69.59,0,3.0,2.0,1,0.0,4.0,3.060,102.066667,0,...,-0.3987,-1.2536,0.2386,-1.2347,-0.6900,1.8736,-0.0026,-0.2210,-0.7745,-0.7978
1454,75.28,0,3.0,2.0,1,0.0,9.0,3.050,71.633333,0,...,-1.1956,-0.1145,-0.9339,1.7156,0.0250,0.4448,0.1905,0.5219,0.8057,2.0751
248,74.58,0,1.0,3.0,1,13.0,6.0,6.086,61.100000,0,...,-0.2372,0.4518,-0.6113,-0.5981,-0.9868,-0.7854,0.1252,0.3379,-1.5421,0.0634
